# import libraries


In [10]:
import tensorflow as tf
import tensorflow.keras.layers as Layers
import numpy as np
import matplotlib.pyplot as plt

# define parameters

In [11]:
LOSS = 'binary_crossentropy'
OPTIMIZER = tf.keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=6e-8)
METRICS = 'accuracy'

INPUT_LENGTH = 128

EPOCHS = 10

IMAGE_SHAPE = (28, 28, 3)

# load data

In [12]:
input_data = tf.keras.datasets.mnist.load_data()[0][0]
print(input_data.shape)

IsADirectoryError: [Errno 21] Is a directory: '/media/Git/Datasets/mnist/'

# define model

### generator

In [ ]:
def generator():
    dim = INPUT_SHAPE//4
    depth = 256
    model = tf.keras.models.Sequential()
    model.add(Layers.Dense(dim*dim*depth, input_dim=INPUT_LENGTH, activation='relu'))
    model.add(Layers.Reshape((dim, dim, depth)))
    model.add(Layers.Conv2DTranspose(depth//4, 3, activation='relu', padding='same'))
    model.add(Layers.UpSampling2D(2))
    model.add(Layers.Conv2DTranspose(depth//16, 3, activation='relu'))
    model.add(Layers.UpSampling2D(2))
    model.add(Layers.Conv2DTranspose(depth//64, 3, activation='relu'))
    model.add(Layers.Conv2DTranspose(3, 3, activation='relu'))
    model.summary()
    return model

### discriminator

In [ ]:
def discriminator(model):
    model = tf.keras.models.Sequential()
    model.add(Layers.Conv2D(128, 3, activation='relu', input_shape=INPUT_SHAPE))
    model.add(Layers.Dropout(0.2))
    model.add(Layers.Conv2D(128, 3, activation='relu'))
    model.add(Layers.Dropout(0.2))
    model.add(Layers.Conv2D(128, 3, activation='relu'))
    model.add(Layers.Dropout(0.2))
    model.add(Layers.MaxPooling2D(2))
    model.add(Layers.Conv2D(128, 3, activation='relu'))
    model.add(Layers.Flatten())
    model.add(Layers.Dense(128, activation='relu'))
    model.add(Layers.Dense(1, activation='sigmoid'))
    model.summary()
    return model

### discriminator model

In [ ]:
def disc_model():
    model = tf.keras.models.Sequential()
    model.add(discriminator())
    model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=METRICS)
    return model

### adversarial model

In [ ]:
def adversarial_model():
    model = tf.keras.models.Sequential()
    model.add(generator())
    model.add(discriminator)
    model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=METRICS)
    return model

# Full model

In [ ]:
gen = generator()
discr = disc_model()
advr = adversarial_model() 

In [ ]:
def get_noise(batch_size):
    return np.random.uniform(-1.0, 1.0, size=[batch_size, INPUT_LENGTH])

In [ ]:
def get_pos_data(batch_size):
    return input_data[np.random.int(input_data[0], batch_size), :, :, :]

In [ ]:
def get_neg_data(batch_size):
    images = gen.predict(get_noise(batch_size))
    return images

In [ ]:
def get_train_labels(batch_size):
    labels = np.ones([2*batch_size, 1])
    labels[batch_size:] = 0
    return labels

# Train model

In [ ]:
def train_discr():
    data_pos = get_pos_data(BATCH_SIZE)
    data_neg = get_neg_data(BATCH_SIZE)
    train_data = np.append(data_pos, data_neg, axis=0)
    train_labels = get_train_labels(BATCH_SIZE)
    return discr.train_on_batch(train_data, train_labels)

In [ ]:
def train_advr():
    noise = get_noise(BATCH_SIZE)
    labels = np.ones([BATCH_SIZE, 1])
    return advr.train_on_batch(noise, labels)

In [ ]:
def train_GAN():
    for e in range(EPOCHS):
        loss_discr = train_discr()
        loss_advr = train_advr()
        log_mesg = "%d: [D loss: %f, acc: %f]" % (i, loss_discr[0], loss_discr[1])
        log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, loss_advr[0], loss_advr[1])
        print(log_mesg)

In [ ]:
train_GAN()

# Generate new images

In [ ]:
num_images = 10
noise = get_noise(num_images)
generated_images = gen.predict(noise)

# View generated images

In [ ]:
fig, axes = plt.subplots((2, num_images/2))
for i in range(num_images):
    axes[i/2, i%2].imshow(generated_images[i])